In [1]:
!pip install wikidata
!pip install networkx
!pip install pyvis

import time
import os.path
import pickle

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from pyvis.network import Network
from wikidata.client import Client
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from collections import defaultdict

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.4 MB/s eta 0:00:00


In [ ]:
!pip install datasets
from datasets import load_dataset, DatasetDict, concatenate_datasets
from huggingface_hub import login
token = ""
login(token=token)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [3]:
dataset = load_dataset('sapienzanlp/nlp2025_hw1_cultural_dataset')
# train, test = dataset["train"], dataset["validation"]
full_dataset = concatenate_datasets([dataset["train"], dataset["validation"]])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/946k [00:00<?, ?B/s]

valid.csv:   0%|          | 0.00/45.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6251 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/300 [00:00<?, ? examples/s]

In [4]:
#Import of stuff
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [5]:
full_dataset_cleaned = [x.split("/")[-1] for x in full_dataset["item"]]

Levels extraction

In [6]:
client = Client()

def fetch_entity(qid):
    try:
        entity = client.get(qid, load=True)
        claims = entity.attributes.get("claims", {})
        return qid, extract_properties(claims)
    except Exception as e:
        print(f"Error on {qid}: {e}")
        return qid, {}

def extract_properties(claims):
    result = {}
    for prop, claim_list in claims.items():
        values = [
            claim.get("mainsnak", {}).get("datavalue", {}).get("value")
            for claim in claim_list
            if claim.get("mainsnak", {}).get("datavalue", {}).get("value") is not None
        ]
        if values:
            result[prop] = values
    return result

In [8]:
level1_file = "/content/drive/MyDrive/SocioEmbeddings/data/text_graph/graph/level1_full_file.plk"

def extract_level1(qid_list, save_path):
    if os.path.isfile(save_path):
        with open(save_path, "rb") as f:
            data = pickle.load(f)
        print(f"Level 1 already loaded from file. ({len(data)} items)")
        return data

    print("Extracting Level 1...")
    data = {}
    with ThreadPoolExecutor() as executor:
        futures = {executor.submit(fetch_entity, qid): qid for qid in qid_list}
        for future in tqdm(as_completed(futures), total=len(futures)):
            qid, prop_vals = future.result()
            data[qid] = prop_vals

    with open(save_path, "wb") as f:
        pickle.dump(data, f)
    print(f"Level 1 completed and saved. ({len(data)} items)")
    return data

# Level 1 exectuion
level1_values = extract_level1(full_dataset_cleaned, level1_file)


Level 1 already loaded from file. (6551 items)


In [9]:
level2_file = "/content/drive/MyDrive/SocioEmbeddings/data/text_graph/graph/level2_full_file.pkl"

def extract_level2(level1_dict, save_path):
    if os.path.isfile(save_path):
        with open(save_path, "rb") as f:
            data = pickle.load(f)
        print(f"Level 2 already loaded from file. ({len(data)} items)")
        return data

    candidate_qids = {
        val["id"]
        for prop_dict in level1_dict.values()
        for values in prop_dict.values()
        for val in values
        if isinstance(val, dict) and "id" in val
    }

    print(f"Extracting Level 2 from {len(candidate_qids)} unique entity values...")
    data = {}
    with ThreadPoolExecutor() as executor:
        futures = {executor.submit(fetch_entity, qid): qid for qid in candidate_qids}
        for future in tqdm(as_completed(futures), total=len(futures)):
            qid, prop_vals = future.result()
            if prop_vals:
                data[qid] = prop_vals

    with open(save_path, "wb") as f:
        pickle.dump(data, f)
    print(f"Level 2 completed and saved. ({len(data)} items)")
    return data

# Lvl 2 exectuion
level2_values = extract_level2(level1_values, level2_file)


Level 2 already loaded from file. (44050 items)



* 2 dictionaries:
  * level1_values: item → {property: [values]}
  * level2_values: entity → {property: [values]}

* Type of nodes:
 * item
 * property
 * entity

* Type of edges:
 * item → property (has_prop)
 * property → entity (has_val)
 * entity → property (has_prop_lvl2)

In [10]:
# This code works only if there's a gpu because it's only for cuda
!pip install torch==2.0.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install torch-geometric


import torch
from torch_geometric.data import Data
from torch_geometric.nn import Node2Vec


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 571.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 72.8 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other 


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

In [11]:
from torch_geometric.data import HeteroData

def build_hetero_graph(level1_values, level2_values):
    data = HeteroData()

    item_nodes = list(level1_values.keys())
    property_nodes = set()
    entity_nodes = set()

    # from level1
    for item, props in level1_values.items():
        for prop, values in props.items():
            property_nodes.add(prop)
            for val in values:
                if isinstance(val, dict) and 'id' in val:
                    entity_nodes.add(val['id'])

    # from level2
    for ent, props in level2_values.items():
        entity_nodes.add(ent)
        for prop in props:
            property_nodes.add(prop)

    # QID/PID mapping
    item2idx = {qid: i for i, qid in enumerate(item_nodes)}
    property2idx = {pid: i for i, pid in enumerate(sorted(property_nodes))}
    entity2idx = {qid: i for i, qid in enumerate(sorted(entity_nodes))}

    # 2. these will be the edges
    item_prop_edges = []
    prop_entity_edges = []
    entity_prop_edges = []

    # item → property, property → entity
    for item, props in level1_values.items():
        for prop, values in props.items():
            if prop not in property2idx:
                continue
            item_idx = item2idx[item]
            prop_idx = property2idx[prop]
            item_prop_edges.append((item_idx, prop_idx))

            for val in values:
                if isinstance(val, dict) and 'id' in val:
                    ent_id = val['id']
                    if ent_id in entity2idx:
                        ent_idx = entity2idx[ent_id]
                        prop_entity_edges.append((prop_idx, ent_idx))

    # level 2: entity -> property
    for ent, props in level2_values.items():
        if ent not in entity2idx:
            continue
        ent_idx = entity2idx[ent]
        for prop, values in props.items():
            if prop in property2idx:
                prop_idx = property2idx[prop]
                entity_prop_edges.append((ent_idx, prop_idx))

    # 3. edge_index
    def to_edge_index(edge_list):
        if not edge_list:
            return torch.empty((2, 0), dtype=torch.long)
        src, dst = zip(*edge_list)
        return torch.tensor([src, dst], dtype=torch.long)

    data['item'].num_nodes = len(item2idx)
    data['property'].num_nodes = len(property2idx)
    data['entity'].num_nodes = len(entity2idx)

    # Dummy node features
    data['item'].x = torch.ones((len(item2idx), 1))
    data['property'].x = torch.ones((len(property2idx), 1))
    data['entity'].x = torch.ones((len(entity2idx), 1))

    # Edges
    data['item', 'has_prop', 'property'].edge_index = to_edge_index(item_prop_edges)
    data['property', 'has_val', 'entity'].edge_index = to_edge_index(prop_entity_edges)
    data['entity', 'has_prop_lvl2', 'property'].edge_index = to_edge_index(entity_prop_edges)

    print(f"HeteroData graph created:")
    print(f" - items: {len(item2idx)}")
    print(f" - properties: {len(property2idx)}")
    print(f" - entities: {len(entity2idx)}")
    print(f" - item→prop edges: {len(item_prop_edges)}")
    print(f" - prop→entity edges: {len(prop_entity_edges)}")
    print(f" - entity→prop edges: {len(entity_prop_edges)}")

    return data, item2idx, property2idx, entity2idx



At the end, i will obtain an Heterodata object like this: <br>

```
HeteroData(
  item={
    num_nodes=6551,
    x=[6551, 1],
  },
  property={
    num_nodes=7306,
    x=[7306, 1],
  },
  entity={
    num_nodes=44075,
    x=[44075, 1],
  },
  (item, has_prop, property)={ edge_index=[2, 200575] },
  (property, has_val, entity)={ edge_index=[2, 93005] },
  (entity, has_prop_lvl2, property)={ edge_index=[2, 1344510] }
)
```



In [12]:
level_path = "/content/drive/MyDrive/SocioEmbeddings/data/text_graph/graph/"

with open(f"{level_path}level1_full_file.plk", "rb") as f:
  level1_values = pickle.load(f)

with open(f"{level_path}level2_full_file.pkl", "rb") as f:
  level2_values = pickle.load(f)


In [ ]:
level1_values

Output hidden; open in https://colab.research.google.com to view.

Merging the two levels

In [13]:
display(len(level1_values))
display(len(level2_values))

6551

44050

In [14]:
full_levels = {}

# Add everything from level2 first
for item, props in level2_values.items():
    full_levels[item] = props

# Then overwrite with level1 (if duplicate, this wins)
for item, props in level1_values.items():
    full_levels[item] = props


In [15]:
display(len(full_levels))

49702

# Extracting and Cleaning the level dictionary

item: {properties: [], entities: []}

In [16]:
def level_to_dictionary(level):
  graph_dictionary = {}

  for item in level:
    if(item not in graph_dictionary):

      properties = set()
      entities = set()
      item_key = level[item]

      for prop in item_key:

        prop_value  = item_key[prop]
        properties.add(prop)
        if(isinstance(prop_value[0], dict) and 'id' in prop_value[0]):
          entities.add(prop_value[0]['id'])

      graph_dictionary[item] = {'properties': properties, 'entities': entities}

  return graph_dictionary


In [17]:
def merge_dictionaries(level1, level2):

  merged_dictionary = {}

  for item in level1:

    entities = level1[item]["entities"].copy()
    properties = level1[item]["properties"].copy()

    for entity in level1[item]["entities"]:
      # access level 2 entities
      if(entity in level2):
        if(entity[0] == "Q"):
          entities = level2[entity]["entities"].union(entities)
        properties = level2[entity]["properties"].union(properties)

    merged_dictionary[item] = {"properties": list(properties), "entities": list(entities)}

  for item in merged_dictionary:
    entities = merged_dictionary[item]["entities"].copy()
    properties = merged_dictionary[item]["properties"].copy()

    merged_dictionary[item]["entities"] = [entity for entity in entities if entity[0] == "Q"]

    # Keep only valid properties
    merged_dictionary[item]["properties"] = [prop for prop in properties if prop[0] == "P"]
  return merged_dictionary

In [31]:
graph_dictionary_path = "/content/drive/MyDrive/SocioEmbeddings/data/text_graph/graph/full_dictionary_lv1lv2"

if(not os.path.isfile(graph_dictionary_path)):

  level1_dic = level_to_dictionary(level1_values)
  level2_dic = level_to_dictionary(level2_values)

  full_dictionary = merge_dictionaries(level1_dic, level2_dic)

  with open(graph_dictionary_path+".plk", "wb") as f:
    pickle.dump(full_dictionary, f)

else:
  with open(graph_dictionary_path+".plk", "rb") as f:
    full_dictionary = pickle.load(f)

In [33]:
graph_path = "/content/drive/MyDrive/SocioEmbeddings/data/text_graph/graph/full_graph_file.plk"
item2idx_path = "/content/drive/MyDrive/SocioEmbeddings/data/text_graph/graph/item2idx.plk"
property2idx_path = "/content/drive/MyDrive/SocioEmbeddings/data/text_graph/graph/property2idx.plk"
entity2idx_path = "/content/drive/MyDrive/SocioEmbeddings/data/text_graph/graph/entity2idx.plk"

if os.path.isfile(graph_path) and os.path.isfile(item2idx_path) and os.path.isfile(property2idx_path) and os.path.isfile(entity2idx_path):
  with open(graph_path, "rb") as f:
    data = pickle.load(f)
    print("loaded graph")

    with open(item2idx_path, "rb") as f:
      item2idx = pickle.load(f)
      print("loaded item2idx")

    with open(property2idx_path, "rb") as f:
      property2idx = pickle.load(f)
      print("loaded property2idx")

    with open(entity2idx_path, "rb") as f:
      entity2idx = pickle.load(f)
      print("loaded entity2idx")
else:
  print("Creation of the graph")
  data, item2idx, property2idx, entity2idx = build_hetero_graph(level1_values, level2_values)
  with open(graph_path, "wb") as f:
    pickle.dump(data, f)
    print("saved data")

  with open(item2idx_path, "wb") as f:
    pickle.dump(item2idx, f)
    print("saved data")

  with open(property2idx_path, "wb") as f:
    pickle.dump(property2idx, f)
    print("saved data")

  with open(entity2idx_path, "wb") as f:
    pickle.dump(entity2idx, f)
    print("saved data")


loaded graph
loaded item2idx
loaded property2idx
loaded entity2idx


In [34]:
print(data['item', 'has_prop', 'property'].edge_index.shape)
print(data['property', 'has_val', 'entity'].edge_index.shape)
print(data['entity', 'has_prop_lvl2', 'property'].edge_index.shape)

torch.Size([2, 200575])
torch.Size([2, 93005])
torch.Size([2, 1344510])


In [20]:
#Functions to navigate and explore the Heterodata object. item "Q280375". More deep yuo go, more time it requires

#1.item -> property
item_qid = "Q280375"
item_id = item2idx[item_qid]
edge_item_prop = data['item', 'has_prop', 'property'].edge_index   #basically is path item->property trough the edge "has_prop"
property_ids = edge_item_prop[1][edge_item_prop[0] == item_id]

idx2property = {v: k for k, v in property2idx.items()}
properties = [idx2property[i.item()] for i in property_ids]
print(f"[1] Item {item_qid} connects to these properties:", properties)

#2.property -> entity (for each property find before)
edge_prop_entity = data['property', 'has_val', 'entity'].edge_index  #property to edge trough has_val edge
idx2entity = {v: k for k, v in entity2idx.items()}

entities = set()
for prop_id in property_ids:
    target_entities = edge_prop_entity[1][edge_prop_entity[0] == prop_id]
    entities.update(idx2entity[e.item()] for e in target_entities)

print(f"[2] These properties connect to entities:", list(entities))

# 3. entity -> property (lelvel 2 of the graph)
edge_entity_prop = data['entity', 'has_prop_lvl2', 'property'].edge_index

entity2prop = defaultdict(list)
for entity in entities:
    if entity in entity2idx:
        ent_id = entity2idx[entity]
        connected_props = edge_entity_prop[1][edge_entity_prop[0] == ent_id]
        props = [idx2property[p.item()] for p in connected_props]
        entity2prop[entity] = props

print(f"[3] Each entity connects to properties (level 2):")
for ent, props in entity2prop.items():
    print(f"    {ent} → {props}")


[1] Item Q280375 connects to these properties: ['P17', 'P18', 'P31', 'P131', 'P1459', 'P625', 'P1435', 'P6375', 'P7959', 'P276', 'P646', 'P2044', 'P281', 'P12485', 'P12749']
[2] These properties connect to entities: ['Q175331', 'Q54935786', 'Q1460219', 'Q1751212', 'Q99', 'Q3258', 'Q639669', 'Q110706912', 'Q1731', 'Q15900616', 'Q41253', 'Q2526135', 'Q4225', 'Q31855', 'Q241450', 'Q1344963', 'Q29943', 'Q476300', 'Q1192543', 'Q3664326', 'Q55304', 'Q1074523', 'Q1034198', 'Q2018526', 'Q1189816', 'Q1200701', 'Q160742', 'Q270791', 'Q2717220', 'Q1022', 'Q485445', 'Q52840', 'Q86135347', 'Q930164', 'Q82346275', 'Q1080794', 'Q69062', 'Q11016', 'Q1449648', 'Q62127981', 'Q840482', 'Q23755128', 'Q6055843', 'Q3325736', 'Q189296', 'Q67532100', 'Q12819564', 'Q7835189', 'Q476028', 'Q12155', 'Q213', 'Q846882', 'Q15711797', 'Q938154', 'Q131734', 'Q1759911', 'Q19479037', 'Q4502119', 'Q38944437', 'Q22922', 'Q16', 'Q280458', 'Q14208553', 'Q580689', 'Q49261', 'Q2625877', 'Q29940414', 'Q156031', 'Q11514315', 'Q

In [ ]:
#IT'S BIG SO DON'T RUN IT, IT'S JUST TO SHOW HOW IT WORKS BUT IT IS VERY EXPENSIVE

# # 4. chain: item -> prop -> entity
# print(f"[4] Full chain: item → prop → entity")
# for prop_id in property_ids:
#     pid = idx2property[prop_id.item()]
#     target_entities = edge_prop_entity[1][edge_prop_entity[0] == prop_id]
#     entity_qids = [idx2entity[e.item()] for e in target_entities]
#     print(f"  {item_qid} → {pid} → {entity_qids}")

# # 5. Full chain: item -> prop -> entity -> prop
# print(f"[5] Full chain: item → prop → entity → prop")
# for prop_id in property_ids:
#     pid = idx2property[prop_id.item()]
#     target_entities = edge_prop_entity[1][edge_prop_entity[0] == prop_id]
#     for ent_id in target_entities:
#         eqid = idx2entity[ent_id.item()]
#         if eqid in entity2idx:
#             connected_props = edge_entity_prop[1][edge_entity_prop[0] == ent_id]
#             prop2 = [idx2property[p.item()] for p in connected_props]
#             print(f"  {item_qid} → {pid} → {eqid} → {prop2}")



In [21]:
data["item"]

{'num_nodes': 6551, 'x': tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [28]:
def hetero_to_homogeneous(data, item2idx, property2idx, entity2idx):
    all_edges = []

    global_dict = {}
    for item_qid, idx in item2idx.items():
        global_dict[f"{item_qid}"] = idx

    # "property:<property_id>" -> index (offset by the number of items)
    for prop_qid, idx in property2idx.items():
        global_dict[f"{prop_qid}"] = len(data['item'].x) + idx

    # "entity:<entity_id>" -> index (offset by the number of items and properties)
    for entity_qid, idx in entity2idx.items():
        global_dict[f"{entity_qid}"] = len(data['item'].x) + len(data['property'].x) + idx

    #reverse mapping: index -> node for quick look-up later
    for idx, item_qid in enumerate(item2idx):
        global_dict[f"{idx}"] = f"{item_qid}"

    for idx, prop_qid in enumerate(property2idx):
        global_dict[f"{len(data['item'].x) + idx}"] = f"{prop_qid}"

    for idx, entity_qid in enumerate(entity2idx):
        global_dict[f"{len(data['item'].x) + len(data['property'].x) + idx}"] = f"{entity_qid}"

    #extract all connections from edges
    for rel in data.edge_types:
        if 'edge_index' not in data[rel]:
          continue  # avoid connections without edges
        edge_index = data[rel]['edge_index']

        src_type, _, dst_type = rel
        src_offset = 0
        dst_offset = 0

        # define offset from each node
        if src_type == 'property':
            src_offset = len(data['item'].x)
        elif src_type == 'entity':
            src_offset = len(data['item'].x) + len(data['property'].x)

        if dst_type == 'property':
            dst_offset = len(data['item'].x)
        elif dst_type == 'entity':
            dst_offset = len(data['item'].x) + len(data['property'].x)

        #offset from starting point
        src = edge_index[0] + src_offset
        dst = edge_index[1] + dst_offset
        all_edges.append(torch.stack([src, dst], dim=0))

    #concatenate all edges
    unified_edge_index = torch.cat(all_edges, dim=1)

    total_nodes = len(data['item'].x) + len(data['property'].x) + len(data['entity'].x)
    x = torch.ones((total_nodes, 1))

    # buil homogeneous graph
    homog_data = Data(x=x, edge_index=unified_edge_index)
    return homog_data, global_dict



In [44]:
homogeneous_graph_path = "/content/drive/MyDrive/SocioEmbeddings/data/text_graph/graph/homogeneous_graph_file.plk"
graph_dict_path = "/content/drive/MyDrive/SocioEmbeddings/data/text_graph/graph/global_dict.plk"

if os.path.isfile(homogeneous_graph_path) and os.path.isfile(graph_dict_path):
  with open(homogeneous_graph_path, "rb") as f:
    homog_data = pickle.load(f)
    print("Homogeneous graph loaded")

    with open(graph_dict_path, "rb") as f:
      global_dict = pickle.load(f)
      print("Global Dictionary loaded")
else:
  print("Creation of the homogeneous graph")
  homog_data, global_dict = hetero_to_homogeneous(data, item2idx, property2idx, entity2idx)
  with open(homogeneous_graph_path, "wb") as f:
    pickle.dump(homog_data, f)
    print("Homogeneous graph created and saved")
  with open(graph_dict_path, "wb") as f:
    pickle.dump(global_dict, f)
    print("Dictionary created and saved")

print(f"Homogeneous graph with {homog_data.num_nodes} nodes and {homog_data.num_edges} edges.")

Creation of the homogeneous graph
Homogeneous graph created and saved
Dictionary created and saved
Homogeneous graph with 57932 nodes and 1638090 edges.


In [43]:
from torch_geometric.utils import to_undirected

homog_data.edge_index = to_undirected(homog_data.edge_index)


Now that we have an homogeneous graph, we can use Node2Vec on our graph. We had to do it because Node2Vec works only with one type of node and one type of edge

In [45]:
node2vec = Node2Vec(
    edge_index=homog_data.edge_index,
    embedding_dim=300,
    walk_length=10,
    context_size=5,
    walks_per_node=2,
    num_negative_samples=10,
    p=1,
    q=0.999,
    sparse=True
)

In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
node2vec = node2vec.to(device)
loader = node2vec.loader(batch_size=32, shuffle=True)
optimizer = torch.optim.SparseAdam(list(node2vec.parameters()), lr=0.01)
epochs = 20

In [ ]:
def train():
    node2vec.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = node2vec.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

for epoch in range(epochs):
    loss = train()
    print(f"Epoch: {epoch:02d}, Loss: {loss:.4f}")


embedding = node2vec.embedding.weight.detach().cpu()
print(f"Embedding shape: {embedding.shape}")
torch.save(embedding, "/content/drive/MyDrive/SocioEmbeddings/data/graph_embedding.pt")